# Default and Custom User Model

## Django предоставляет дефолтную модель user

https://docs.djangoproject.com/en/3.2/topics/auth/default/#user-objects

Использование:
```python
from django.contrib.auth.models import User

user = User.objects.create_user('john', 'lennon@thebeatles.com', 'johnpassword')
```

## А как расширять и модифировать?

https://habr.com/en/post/313764/

- Ниже мы разберем стратегию расширения, существующей модели
- **ДЗ:** а в домашнем задании, надо будет модифицировать и удалить ненужные поля

## Создадим новый модуль users


```shell
python manage.py startapp users
```

In [ ]:
# tms_django_lessons/settings.py
INSTALLED_APPS = [
    ...
    'users',
]

## Создадим модель Profile для дополнительных полей


In [ ]:
# users/models.py
from django.db import models
from django.contrib.auth.models import User


class Profile(models.Model):
    user = models.OneToOneField(User, on_delete=models.CASCADE)
    bio = models.TextField(max_length=500, blank=True)
    location = models.CharField(max_length=30, blank=True)
    birth_date = models.DateField(null=True, blank=True)

## Протестируем

- добавим миграции
- попробуем создать пользователя
```python
user = User.objects.create_user('some_user', 'some_user@some_user.com', '123123abc')
print(user.profile)
```

Note: как оказалось профиль для дополнительных полей не создался

## Добавим триггер на автоматическое создание профиля при создании пользователя

https://docs.djangoproject.com/en/3.2/topics/signals/


In [ ]:
# users/models.py
...
from django.db.models.signals import post_save
from django.dispatch import receiver

...

@receiver(post_save, sender=User)
def create_user_profile(sender, instance, created, **kwargs):
    if created:
        Profile.objects.create(user=instance)

## Протестируем

- добавим миграции
- попробуем создать пользователя
```python
user = User.objects.create_user('test', 'bla@bla.com', '123123abc')
print(user.profile)
```